In [4]:
import boto3, os, glob
import pandas as pd
import datetime as dt
from dateutil import parser

In [5]:
def build_id_int(x):
    d1 = parser.parse(x)
    d2 = (d1 - dt.datetime(1970, 1, 1)).total_seconds()
    return int(d2)

In [6]:
session = boto3.Session(profile_name='default')

In [7]:
s3 = boto3.resource('s3')

In [8]:
dev_s3_client = session.client('s3')

In [5]:
paginator = dev_s3_client.get_paginator('list_objects')

In [6]:
operation_parameters = {'Bucket': 'bat-2020',
                        'Prefix': 'bat_listings'}

In [7]:
page_iterator = paginator.paginate(**operation_parameters)

In [8]:
filelist = []
for page in page_iterator:
    for item in page['Contents']:
        if item['Key'][:3] == 'bat':
            filelist.append(item['Key'])

In [9]:
len(filelist)

946

In [10]:
inpath = '/home/ubuntu/projects/bat-2020-data/temp-in/'
outpath = '/home/ubuntu/projects/bat-2020-data/temp-out/'

In [14]:
for file in filelist:
    downfilename = inpath + file
    upfilename = outpath + file
    downfile = s3.meta.client.download_file(Bucket='bat-2020',
                                          Key = file,
                                          Filename = downfilename)
    df = pd.read_csv(downfilename)
    df = df.rename(columns = {'Unnamed: 0': 'fid'})
    df.listing_excerpt = [df.iloc[i].listing_excerpt.replace('\n', ' ') for i in range(len(df))]
    df.fid = [str(df.iloc[i].fid) + '_' + str(build_id_int(df.iloc[i].scrape_ts)) for i in range(len(df))]
    df.to_csv(upfilename, index=False)
    upfile = s3.meta.client.upload_file(Filename = upfilename,
                                        Bucket='bat-2020',
                                        Key = file)
    del(df)
    del(downfilename)
    del(upfilename)

In [ ]:
newfile = s3.meta.client.download_file(Bucket='bat-2020', 
                                      Key = 'bat_listings_output_2020-10-12_21-11-19.csv',
                                      Filename = '/home/ubuntu/projects/bat-2020-data/temp/bat_listings_output_2020-10-12_21-11-19.csv')

In [ ]:
df.iloc[25]['scrape_ts']

In [ ]:
id_int

In [ ]:
build_id_int(df.iloc[23].scrape_ts)

In [ ]:
df = pd.read_csv('/home/ubuntu/projects/bat-2020-data/temp-in/bat_listings_output_2020-10-12_21-11-19.csv')
df = df.rename(columns = {'Unnamed: 0': 'fid'})
df.listing_excerpt = [df.iloc[i].listing_excerpt.replace('\n', ' ') for i in range(len(df))]
df.fid = [str(df.iloc[i].fid) + '_' + str(build_id_int(df.iloc[i].scrape_ts)) for i in range(len(df))]
df.to_csv('/home/ubuntu/projects/bat-2020-data/temp-out/bat_listings_output_2020-10-12_21-11-19.csv', index=False)

In [ ]:
df.head()

In [ ]:
## test out method for changing files

In [ ]:
directory = '../../bat-2020-data/'
allfiles = list(glob.glob(os.path.join(directory, '*.csv')))

In [ ]:
import pandas as pd
help(pd.DataFrame.rename)

In [ ]:
df = pd.read_csv(allfiles[241])

In [ ]:
df = df.rename(columns={"Unnamed: 0":"fid"})

In [ ]:
df.columns

In [ ]:
for file in allfiles:
    df = pd.read_csv(file)
    df = df.drop('Unnamed: 0', axis='columns')
    df.to_csv(file, index=False)

In [ ]:
print('fid'+str(allfiles[2]))

In [ ]:
for file in allfiles:
    dev_s3_client.upload_file(file, 'bat-2020', file.split('/')[3])

In [ ]:
! zip -9 ../../bat-2020-data/*.csv ../../bat-2020-data/zipfile.csv

In [ ]:
!zip -9 ../../bat-2020-data/zipfile.zip ../../bat-2020-data/*.csv

In [ ]:
!rm -f ../../bat-2020-data/*.csv

### modify files on RPi

In [3]:
import os, glob
import pandas as pd

In [9]:
directory = '../../bat-2020-data/'
allfiles = list(glob.glob(os.path.join(directory, 'bat_listings_output*.csv')))

In [10]:
allfiles

['../../bat-2020-data/bat_listings_output_2021-04-17_21-31-20.csv',
 '../../bat-2020-data/bat_listings_output_2021-04-03_21-34-18.csv',
 '../../bat-2020-data/bat_listings_output_2021-03-25_12-34-17.csv',
 '../../bat-2020-data/bat_listings_output_2021-03-26_00-34-16.csv',
 '../../bat-2020-data/bat_listings_output_2021-03-29_12-34-17.csv',
 '../../bat-2020-data/bat_listings_output_2021-03-28_00-34-19.csv',
 '../../bat-2020-data/bat_listings_output_2021-04-16_18-34-19.csv',
 '../../bat-2020-data/bat_listings_output_2021-04-01_18-34-16.csv',
 '../../bat-2020-data/bat_listings_output_2021-04-15_00-34-21.csv',
 '../../bat-2020-data/bat_listings_output_2021-03-30_18-34-17.csv',
 '../../bat-2020-data/bat_listings_output_2021-04-12_00-34-20.csv',
 '../../bat-2020-data/bat_listings_output_2021-04-09_12-34-17.csv',
 '../../bat-2020-data/bat_listings_output_2021-03-28_21-34-18.csv',
 '../../bat-2020-data/bat_listings_output_2021-04-05_12-34-20.csv',
 '../../bat-2020-data/bat_listings_output_2021-0

In [11]:
allfiles[14].split('/')[-1]

'bat_listings_output_2021-04-05_15-34-18.csv'

In [13]:
for file in allfiles:
    df = pd.read_csv(file)
    fileonly = file.split('/')[-1]
    df = df.rename(columns = {'Unnamed: 0': 'fid'})
    df.listing_excerpt = [df.iloc[i].listing_excerpt.replace('\n', ' ') for i in range(len(df))]
    df.fid = [str(df.iloc[i].fid) + '_' + str(build_id_int(df.iloc[i].scrape_ts)) for i in range(len(df))]
    df.to_csv('/home/ubuntu/projects/bat-2020-data/temp-out'+fileonly, index=False)
    upfile = s3.meta.client.upload_file(Filename = file,
                                        Bucket='bat-2020',
                                        Key = fileonly)
    print('uploaded file: {0}'.format(file))
    del(df)

uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-17_21-31-20.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-03_21-34-18.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-25_12-34-17.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-26_00-34-16.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-29_12-34-17.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-28_00-34-19.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-16_18-34-19.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-01_18-34-16.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-15_00-34-21.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-30_18-34-17.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-12_00-34-20.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-09_12-34-17.csv
uploaded file: ../../bat-2020-data/bat_listings_outp

uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-30_21-34-17.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-01_09-34-17.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-06_09-34-17.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-05_21-34-18.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-25_00-34-16.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-24_15-34-17.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-02_15-34-17.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-27_21-34-16.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-14_18-34-21.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-06_18-34-16.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-04-07_12-34-20.csv
uploaded file: ../../bat-2020-data/bat_listings_output_2021-03-28_15-34-18.csv
uploaded file: ../../bat-2020-data/bat_listings_outp

In [14]:
for file in allfiles:
    os.remove(file)